In [1]:
import os
os.chdir("../")
%pwd

'/home/ajith/projects/Spaceship_Titanic_MLOps_Project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_learning_rate: float
    params_features: int

In [3]:
from titanicSpaceShip.constants import *
from titanicSpaceShip.utils.common import read_yaml, create_directories


In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
                root_dir=Path(config.root_dir),
                base_model_path=Path(config.base_model_path),
                params_learning_rate=self.params.LEARNING_RATE,
                params_features=self.params.FEATURES
        )

        return prepare_base_model_config

In [6]:

import torch.nn as nn

class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_size, 32)
        self.layer2 = nn.Linear(32, 64)
        self.layer3 = nn.Linear(64, 32)
        self.layer4 = nn.Linear(32, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        x = self.relu(x)
        x = self.sigmoid(self.layer4(x))
        return x

In [7]:
import torch

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = Model(self.config.params_features, 1)

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def save_model(path: Path, model: nn.Module):
        # model.save(path)
        torch.save(model, path)

In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2024-01-14 12:28:47,774: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-14 12:28:47,784: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-14 12:28:47,785: INFO: common: created directory at: artifacts]
[2024-01-14 12:28:47,789: INFO: common: created directory at: artifacts/prepare_base_model]
